#MP4

In this assignment you will be performing Semantic Segmentation. We've provided the dataset and some helper code to guide you along.

Reminders:
- When first getting your code to run do not use GPU as this will exhaust your colab resources
- When you're ready to properly test your models, make sure you are connected to a GPU runtime as this does significantly speeds up execution
    - To change your runtime do: **Runtime** --> **Change runtime type** --> under **Hardware accelerator** select **GPU**
    - Note that changing runtime resets your kernel (meaning you will need to rerun cells and local variables will be lost)
    - It also sets this new runtime as the default when you return to this notebook later
- Do not start last minute, these models do take some time to train
- Loading the data takes some time, you should only have to do this once

## Accessing the data

There are multiple ways to work with data in colab.
See this [Colab notebook](https://colab.research.google.com/notebooks/io.ipynb) or this [StackOverflow post](https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive) for more details.

Once you've mounted your drive you can see your entire drive file structure by clicking the "Files" tab on the left.

**If you wish to work locally you can ignore the first two cells, but you will still need to set the appropriate path for your dataset**

In [ ]:
# Note there are other methods to do this
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import os
if not os.path.exists("/content/gdrive/My Drive/CS_498_MP4"):
    os.makedirs("/content/gdrive/My Drive/CS_498_MP4")
os.chdir("/content/gdrive/My Drive/CS_498_MP4")

# TODO: make sure to specify the right dataset path here
DATASET_PATH = 'data/sbd/'
# DATASET_PATH = '/content/gdrive/MyDrive/CS_498_MP4/data/sbd/'

In [ ]:
import glob
import os
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, average_precision_score
import random

from PIL import Image
import torch
from torch import nn
from torch.utils import data
import torchvision
from torchvision import models
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torch.optim as optim

## Dataset (Q1)

Here we define a class (pytorch Dataset) for accessing data. This allows us to perform transformations on the data (data augmentation) as we access it. Pytorch dataloaders take in a dataset and conventiently deal with the overhead of looping through it in batches. Creating such datasets/loaders significantly simplifies our training code later on.

**PDF: In your pdf visualize the same image (your choice which) a couple times to demonstrate your transformations**



In [ ]:
class SegmentationDataset(data.Dataset):
    """
    Data loader for the Segmentation Dataset. If data loading is a bottleneck, 
    you may want to optimize this in for faster training. Possibilities include
    preloading all images and annotations into memory before training, to limit delays due to disk reads.
    """
    def __init__(self, split="train", preload=True, data_dir=DATASET_PATH, transform=False):
        assert(split in ["train", "val", "test"])
        self.img_dir = os.path.join(data_dir, split)
        self.classes = []
        with open(os.path.join(data_dir, 'classes.txt'), 'r') as f:
            for l in f:
                self.classes.append(l.rstrip())
        self.n_classes = len(self.classes)
        self.split = split
        self.data = glob.glob(self.img_dir + '/*.jpg') 
        self.data = [os.path.splitext(l)[0] for l in self.data]
        self.transform = transform
        self.preload = preload
        # preload data
        if preload:
            self.images = [Image.open(self.data[index] + '.jpg') for index in range(len(self.data))]
            self.ground_truth = [Image.open(self.data[index] + '.png') for index in range(len(self.data))]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.preload:
            img = self.images[index]
            gt = self.ground_truth[index]
        else:
            img = Image.open(self.data[index] + '.jpg')
            gt = Image.open(self.data[index] + '.png')

        # Question 1: data augmentation
        # hint: how does transforming the image affect the ground truth?
        # Note: your transformation should not change the image dimensions    
        if self.transform:
            # Your code
            # -------------------------
            pass
            # -------------------------

        img = ToTensor()(img)
        gt = torch.LongTensor(np.array(gt)).unsqueeze(0)

        return img, gt

In [ ]:
dataset = SegmentationDataset(split="train", preload=True, data_dir=DATASET_PATH, transform=True)

In [ ]:
# TODO: set the batch size, when running experiments later you should try different batch sizes
training_batch_size = None
dataloader = data.DataLoader(dataset, batch_size=training_batch_size, shuffle=True, num_workers=2, drop_last=True)

In [ ]:
val_dataset = SegmentationDataset(split="val", preload=True, data_dir=DATASET_PATH, transform=False)

In [ ]:
val_dataloader = data.DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, drop_last=False)

In [ ]:
def view_image(idx):
    img = dataset[idx]
    _, axes = plt.subplots(1,2)
    axes[0].imshow(np.swapaxes(np.swapaxes(img[0], 0, 2), 0, 1))
    axes[1].imshow(img[1][0])
    plt.show()

In [ ]:
# You might want to look at a bunch of different images to get a feel for your data
view_image(random.randint(0, len(dataset)))

## Simple Baseline (Q2)

This is a trivial semantic segmentor. For each pixel location it computes the 
distribution of the class label in the training set and uses that as the 
prediction. In other words, if a pixel is "sky" half the time and "water" the other half in the training data, you should label it as [0.5,0,0,0,0.5,0,0,0,0].

**PDF: in your pdf report the evaluation metrics (from the next question) for this simple baseline. Also visualize the output image of simple_predict (since simple_predict outputs the same segmentation regardless of input you can just report a single image)**

In [ ]:
# Question 2
# Output shape: (num_classes, 224, 288)
def simple_train(num_classes, train_dataloader):
    # Your code
    # -------------------------
    model = np.ones((num_classes, 224, 288)) / num_classes
    # -------------------------
    return model

# Output:
#   gt: the ground truth segmentation, shape (dataset_size, 1, 224, 288)
#   preds: the predicted segmentation class probabilities, shape (dataset_size, 9, 224, 288) 
def simple_predict(dataloader, model):
    gts, preds = [], []
    for i, batch in enumerate(tqdm(dataloader)):
        # Your code
        # -------------------------
        pass
        # -------------------------
    return np.array(gts), np.array(preds)

In [ ]:
# our "model" is class frequency, train it then make predictions for the validation set 
class_freq = simple_train(dataset.n_classes, dataloader)
gts, preds = simple_predict(val_dataloader, class_freq)
classes = list(dataset.classes)

In [ ]:
# visualize the output segmentation prediction
plt.imshow(np.argmax(preds[0], axis=0))

## Evaluation Metrics (Q3)

We've implemented mean average precision. Your job is to compute the confusion matrix and IoU for a set of predictions. Namely, fill in the compute_confusion_matrix function.

The **(i,j)**th entry of a confusion matrix computes the number of observations known to be in group **i** and predicted to be in group **j**. You can use [sklearn.metrics.confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) as a reference and sanity check.

IoU is the intersection of the predicted and ground truth segmentation masks divided by their union. Think how these values are related to what you've already computed in the confusion matrix. 

In [ ]:
def segmentation_eval(gts, preds, classes, plot_file_name):
    """
    @param    gts               numpy.ndarray   ground truth labels
    @param    preds             numpy.ndarray   predicted labels
    @param    classes           string          class names
    @param    plot_file_name    string          plot file names
    """
    ious, counts = compute_confusion_matrix(gts, preds)
    aps = compute_ap(gts, preds)
    plot_results(counts, ious, aps, classes, plot_file_name)
    for i in range(len(classes)):
        print('{:>20s}: AP: {:0.2f}, IoU: {:0.2f}'.format(classes[i], aps[i], ious[i]))
    print('{:>20s}: AP: {:0.2f}, IoU: {:0.2f}'.format('mean', np.mean(aps), np.mean(ious)))
    return aps, ious

def plot_results(counts, ious, aps, classes, file_name):
    fig, ax = plt.subplots(1,1)
    conf = counts / np.sum(counts, 1, keepdims=True)
    conf = np.concatenate([conf, np.array(aps).reshape(-1,1), 
                           np.array(ious).reshape(-1,1)], 1)
    conf = conf * 100.
    sns.heatmap(conf, annot=True, ax=ax, fmt='3.0f') 
    arts = [] 
    # labels, title and ticks
    _ = ax.set_xlabel('Predicted labels')
    arts.append(_)
    _ = ax.set_ylabel('True labels')
    arts.append(_)
    _ = ax.set_title('Confusion Matrix, mAP: {:5.1f}, mIoU: {:5.1f}'.format(
      np.mean(aps)*100., np.mean(ious)*100.))
    arts.append(_)
    _ = ax.xaxis.set_ticklabels(classes + ['AP', 'IoU'], rotation=90)
    arts.append(_)
    _ = ax.yaxis.set_ticklabels(classes, rotation=0)
    arts.append(_)
    # fig.savefig(file_name, bbox_inches='tight')
    plt.show()

def compute_ap(gts, preds):
    aps = []
    for i in range(preds.shape[1]):
        ap, prec, rec = calc_pr(gts == i, preds[:,i:i+1,:,:])
        aps.append(ap)
    return aps

def calc_pr(gt, out, wt=None):
    gt = gt.astype(np.float64).reshape((-1,1))
    out = out.astype(np.float64).reshape((-1,1))
    tog = np.concatenate([gt, out], axis=1)*1.
    ind = np.argsort(tog[:,1], axis=0)[::-1]
    tog = tog[ind,:]
    cumsumsortgt = np.cumsum(tog[:,0])
    cumsumsortwt = np.cumsum(tog[:,0]-tog[:,0]+1)
    prec = cumsumsortgt / cumsumsortwt
    rec = cumsumsortgt / np.sum(tog[:,0])
    ap = voc_ap(rec, prec)
    return ap, rec, prec

def voc_ap(rec, prec):
    rec = rec.reshape((-1,1))
    prec = prec.reshape((-1,1))
    z = np.zeros((1,1)) 
    o = np.ones((1,1))
    mrec = np.vstack((z, rec, o))
    mpre = np.vstack((z, prec, z))

    mpre = np.maximum.accumulate(mpre[::-1])[::-1]
    I = np.where(mrec[1:] != mrec[0:-1])[0]+1;
    ap = np.sum((mrec[I] - mrec[I-1])*mpre[I])
    return ap
    
# Question 3: compute the confusion matrix and IoU metrics for each class
# Hint: once you've computed the confusion matrix, IoU is easy
# Note: preds contains class probabilities, convert this to a class prediction
def compute_confusion_matrix(gts, preds):
    # Your code
    IoU = np.zeros((9))
    conf = np.eye(9)
    return IoU, conf

In [ ]:
# Evaluate our trivial segmentor
aps, ious = segmentation_eval(gts, preds, classes, 'cs543-simple-train.pdf')

## Loss function (Q4)

Implement the weighted cross entropy loss. 

You may not call nn.CrossEntropy but can use it as a good reference and sanity check.

**PDF: in your pdf please describe the cross entropy loss. Also explain the purpose of using a weighted loss.**

In [ ]:
def cross_entropy_criterion(predictions, labels, weights):
    # your code
    loss = 0
    return loss

## Training loop (Q5)

Fill in the training loop. We've provided validation code as well as skeleton code for training.

Keep in mind that you need to move data onto the device (GPU) as you cycle through the dataloader

While we've provided you with a skeleton to fill in, you should feel free to modify the visualization code for debugging purposes. For example you might want to print out the loss each iteration instead of once per epoch. Or you might want to compute validation accuracy metrics (like IoU) instead of just validation loss.

**PDF: in your pdf please describe why it is important to consider both validation and training losses simultaneously. When loss stops decreasing, can we change something about the training parameters to continue improving the model?**

In [ ]:
def validate_model(val_loader, model, classes, device, show_matrix=False):
    preds = np.array([]).reshape(0,9,224,288)
    gts = np.array([]).reshape(0,1,224,288)
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs = inputs.to(device)

            outputs = model(inputs).cpu().numpy()
            preds = np.concatenate([preds, outputs], axis=0)
            gts = np.concatenate([gts, labels.numpy()], axis=0)
            
            print("Validating...{}\r".format(100.0*len(preds)/len(val_loader)), end="")
    
    if show_matrix:
        aps, ious = segmentation_eval(gts, preds, classes, 'cs543-simple-val_3.pdf')
    else:
        ious, counts = compute_confusion_matrix(gts, preds)
        aps = compute_ap(gts, preds)
        for i in range(len(classes)):
            print('{:>20s}: AP: {:0.2f}, IoU: {:0.2f}'.format(classes[i], aps[i], ious[i]))
        print('{:>20s}: AP: {:0.2f}, IoU: {:0.2f}'.format('mean', np.mean(aps), np.mean(ious)))

    return preds, gts

# Your goal is to complete this function
def train(model, optimizer, criterion, trainloader, device, valloader = None, epochs=15):
    train_loss_over_epochs = []
    val_loss_over_epochs = []
    plt.ioff()
    fig = plt.figure()
    for epoch in tqdm(range(epochs), total=epochs):
        # running loss is the **average** loss for each item in the dataset during this epoch
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # Your code
            # -------------------------
            pass
            # -------------------------

        train_loss_over_epochs.append(running_loss)
        # Note: it can be more readable to overwrite the previous line - end="\r"
        print('Epoch: {}, training loss: {:.3f}'.format(epoch + 1, running_loss))

        # If you pass in a validation dataloader then compute the validation loss
        if not valloader is None:
            val_loss = 0.0
            with torch.no_grad():
                for data in valloader:
                    # Your code
                    # -------------------------
                    pass
                    # -------------------------
            val_loss_over_epochs.append(val_loss)
            print('Epoch: {}, training loss: {:.3f}'.format(epoch + 1, val_loss))
        
    plt.subplot(2, 1, 1)
    plt.ylabel('Loss')
    plt.plot(np.arange(epochs), train_loss_over_epochs, color='red', label='train')
    if not valloader is None:
        plt.plot(np.arange(epochs), val_loss_over_epochs, color='blue', label='val')
    plt.title('Loss per Epoch')
    plt.xticks(np.arange(epochs, dtype=int))
    plt.grid(True)
    plt.legend()
    plt.show()
    return model



## Model definitions (Q6)

Now create your models. Create one basic Convolutional architecture and one U-Net architecture.

We provide some helpful methods below to compute the size of your next convolutional layer (you can find these formula at TODO).

Some things to keep in mind:
- your basic layer is nn.Conv2D, read its documentation
- for UNet you will also need nn.ConvTranspose2D and Pooling layers
- nn.BatchNorm2d is incredibly helpful between layers
- you can stick to ReLU activations, but are welcome to report results with other activation functions

**PDF: in your pdf please describe your final model architectures. Report the training plots and final accuracy metrics on the validation set for each model. What batch size, learning rate, optimizer did you find works best. Perform a small ablation study: what is the effect of batchnorm on training speed and accuracy? Visualize a few images and their predicted segmentation masks by your UNet model.**

In [ ]:
def conv_out_size(inp_size, kernel_size, dilation, padding, stride):
    return ((inp_size + 2*padding - dilation * (kernel_size - 1) - 1) // stride) + 1

def conv_trans_out_size(inp_size, kernel_size, dilation, padding, stride, out_padding):
    return (inp_size - 1) * stride - 2*padding + dilation * (kernel_size - 1) + out_padding + 1

In [ ]:
class BaseConv(nn.Module):
    def __init__(self):
        super(BaseConv, self).__init__()
        
        # Your code

    def forward(self, x):
        # Your code
        
        return x

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Your code

    def forward(self, x):
        # Your code
        
        return x

### Now we can finally train our models...

In [ ]:
# if runtime has GPU use GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print("Using device:", device)

In [ ]:
# For the weighted cross entropy loss we can compute class weights using our simple baseline
class_freq = simple_train(dataset.n_classes, dataloader)
class_weights = []
for i in range(9):
    class_weights.append(1 / np.mean(class_freq[i, :, :]))
print(class_weights)

### Basic convolutional model training

In [ ]:
# First make the model and put it on the device
base_model = BaseConv().to(device)

# Now define our loss criterion as cross entropy based on your previous code
#criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float).to(device))
criterion = lambda y_pred, y_true: cross_entropy_criterion(y_pred, y_true, class_weights)

# Now make our optimizer for this model
# TODO: pick an optimizer from torch.optim and set the learning rate
lr = None
optimizer = None

In [ ]:
# Now train and validate
# Consider putting this code into a loop, 
# thus alternating between training for some number of epochs and validating

# TODO: how many epochs to train for?
epochs = None
base_model = train(base_model, optimizer, criterion, dataloader, device, epochs=epochs)
preds, gts = validate_model(val_dataloader, base_model, list(dataset.classes), device)

#### UNet model training

In [ ]:
model_unet = UNet().to(device)

# TODO: fill in this code as you did above for the basic convolutional model

Make sure to report the results for both models. 

## Working off a pretrained model (Q7)

Finally, you will now modify a pretrained model (resnet18) and use it as an initialization for training. You should be able to get better results with this model than before.

You can finetune (meaning backpropagate through the resnet layers) or not. You can finetune just some layers and not others. It's up to you.

**PDF: in your pdf report the final accuracy of your model based on a pretrained model. Describe how you used the pretrained model, which features did you extract and why?**

In [ ]:
pretrained_resnet = models.resnet18(pretrained=True)

In [ ]:
class ResnetBasedModel(nn.Module):
    def __init__(self, pretrained_resnet, num_layers_to_remove=0):
        super(ResnetBasedModel, self).__init__()
        # You can, for example, extract the first N layers of the model like this:
        # self.resnet_features = nn.Sequential(*list(pretrained_resnet.children())[:N])

    def forward(self, x):
        return x


In [ ]:
resnet_based_model = ResnetBasedModel(pretrained_resnet).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float).to(device))
optimizer = optim.Adam(resnet_based_model.parameters(), lr=0.001)

resnet_based_model = train(resnet_based_model, optimizer, criterion, dataloader, device, epochs=15)
preds, gts = validate_model(val_dataloader, resnet_based_model, list(dataset.classes), device)

# Test set

Finally we can check evaluation on test set....

**PDF: in your pdf report the results of your best model (this should be based on a pretrained model) on the test dataset.**

In [ ]:
test_dataset = SegmentationDataset(split="test", preload=True, data_dir=DATASET_PATH, transform=False)
test_dataloader = data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2, drop_last=False)